In [13]:
%pip install pandas morfessor 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
#unsupervised

#TODO: Add command line script for training the model

import pandas as pd
import morfessor
import subprocess
import os
morph_length = 5
train = subprocess.run(['morfessor-train.bat', '-s', 'models\m_morfessor.bin','--morph-length', f'{morph_length}', '--traindata-list', 'data\\training_morfessor_cli.txt'])

input_tsv = 'data\\eng.sentence.test.gold.tsv'  #path to the input tsv file
output_tsv = 'outputs\\eng.sentence.test.morfessor_guess.tsv'  #path tot the output tsv file
bi_model = 'models\\m_morfessor.bin'

io = morfessor.MorfessorIO()
model = io.read_binary_model_file(bi_model)



df_guess = pd.read_csv(input_tsv, sep='\t', header=None)
data_guess = df_guess[0].astype(str)
i = 0

#segment on words

for entry in data_guess:
  sent = ''
  j = 0
  words = entry.split() #list of words
  length_sent = len(words)  #number of words

  for word in words:
    list_word = model.viterbi_segment(word)[0]
    k = 0
    length_word = len(list_word)  #number of morphs

    for morph in list_word:
      if k != length_word - 1:  #not last morph
        sent += (morph + ' @@')
        k+=1
      else: #last morph
        if j != length_sent - 1:
          sent += (morph + ' ')
        else :
          sent +=morph
    j+=1

  df_guess[1][i] = sent
  i+=1

df_guess.to_csv(output_tsv, sep='\t', header=None, index = False)

import argparse
args = argparse.Namespace(
    gold="data\\eng.sentence.test.gold.tsv",
    guess="outputs\\eng.sentence.test.morfessor_guess.tsv",
    output="outputs\\output.json",
    category=False 
)
import evaluate, json, os
stats = evaluate.main(args)
model_name = f"morfessor_len_{morph_length}"
new_stats = {"model": model_name}
new_stats.update(stats)
data = {"data": []}
if os.path.exists(args.output):
    with open(args.output, 'r') as output_file:
        data = json.load(output_file)

# Skip adding data point if already present
if not any(item.get("model") == model_name for item in data["data"]):
    data["data"].append(new_stats)
    data["data"] = sorted(data["data"], key=lambda x: x["model"])
    with open(args.output, 'w') as output_file:
        json.dump(data, output_file, indent=4)


<>:10: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
C:\Users\admin\AppData\Local\Temp\ipykernel_7560\1884829732.py:10: SyntaxWarning: invalid escape sequence '\m'
  train = subprocess.run(['morfessor-train.bat', '-s', 'models\m_morfessor.bin','--morph-length', f'{morph_length}', '--traindata-list', 'data\\training_morfessor_cli.txt'])
C:\Users\admin\AppData\Local\Temp\ipykernel_7560\1884829732.py:49: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "co



category: all
distance	2.71
f_measure	81.31
precision	82.78
recall	79.89


In [ ]:
# evaluate_script_path = r'NLP-Praktikum\segmentation_techniques\evaluate.py'

# try:
#   eval = subprocess.run([r'C:\Users\admin\Documents\code\NLP-Praktikum\nlp-venv\Scripts\python.exe', 'evaluate.py', '--guess', 'outputs\\eng.sentence.test.morfessor_guess.tsv', '--gold', 'data\\eng.sentence.test.gold.tsv'] ,
#                          capture_output=True,  
#                          text=True,
#                          check=True            
#   )
#   print(eval.stdout)
# except subprocess.CalledProcessError as e:
#   print(e.stderr)



category: all
distance	6.04
f_measure	59.72
precision	52.90
recall	68.55

